## Object Detection and Localization using R-CNN

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

### downloading module from tensorflow hub

In [2]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1";

detector = hub.load(module_handle).signatures['default']

### function for download and resize

In [3]:
def download_and_resize_image(url, new_width=256, new_height=256):
  """
  Fetches an image online, resizes it and saves it locally.

  Args:
    url (string) -- link to the image
    new_width (int) -- size iin pixels used for resizing the width of the image
    new_height (int) -- size in pixels used for resizing the length of the image

  Returns:
    (string) -- path to saved image
  """

  # create a temporary file ending with ".jpg"
  _, filename = tempfile.mkstemp(suffix=".jpg")

  # opens the given URL
  response = urlopen(url)

  # reads the image fetched from the URL
  image_data = response.read()

  # puts the image data in memory buffer
  image_data = BytesIO(image_data)

  # opens the image
  pil_image = Image.open(image_data)

  # resizes the image. will crop if aspect ratio is different.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)

  # converts to the RGB colorspace
  pil_image_rgb = pil_image.convert("RGB")

  # saves the image to the temporary file created earlier
  pil_image_rgb.save(filename, format="JPEG", quality=90)

  print("Image downloaded to %s." % filename)

  return filename

In [4]:
img_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG";

downloaded_image_path = download_and_resize_image(
    url = img_url,
    new_width = 3872,
    new_height = 2592
)

Image downloaded to /tmp/tmpaubj330c.jpg.


<ipython-input-3-081da85fe049>:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)


### Load_img to convert tensor to image
### run_detector to detect object and draw bounding lines


In [5]:
def load_img(path):
  """
  Loads a JPEG image and converts it to a tensor.

  Args:
    path (string) -- path to a locally saved JPEG image

  Returns:
    (tensor) -- an image tensor
  """

  # read the file
  img = tf.io.read_file(path)

  # convert to a tensor
  img = tf.image.decode_jpeg(img, channels=3)

  return img

def run_detector(detector, path):
  """
  Runs inference on a local files using an object detection model.

  Args:
    detector (model) -- an object detection model loaded from TF hub
    path (string) -- path to an image saved locally
  """

  # load an image tensor from a local file path
  img = load_img(path)

  # add a batch dimension in front of the tensor
  converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  # run inference using the model
  result = detector(converted_img)

  # save the results in a dictionary
  result = {key: value.numpy() for key, value in result.items()}

  # print results
  print("Found %d objects." % len(result["detection_scores"]))

  print(result["detection_scores"])
  print(result["detection_class_entities"])
  print(result["detection_boxes"])

In [ ]:
run_detector(detector, downloaded_image_path)

### Clone the tensorflow models repository

In [7]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3984, done.
remote: Counting objects: 100% (3984/3984), done.
remote: Compressing objects: 100% (3085/3085), done.
remote: Total 3984 (delta 1159), reused 1954 (delta 842), pack-reused 0
Receiving objects: 100% (3984/3984), 49.75 MiB | 25.99 MiB/s, done.
Resolving deltas: 100% (1159/1159), done.


In [9]:
# protoc object_detection/protos/*.proto --python_out=.

!cd models/research/
!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .
!python -m pip install

Could not make proto path relative: object_detection/protos/*.proto: No such file or directory
cp: cannot stat 'object_detection/packages/tf2/setup.py': No such file or directory
ERROR: You must give at least one requirement to install (see "pip help install")
